## 필요 라이브러리 로드

In [171]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape
import ifcopenshell.util.selector
import numpy as np

## ifc 파일 로드

In [172]:
ifc_file = ifcopenshell.open('sample.ifc')
print('ifc_file : ', ifc_file)

ifc_file :  <ifcopenshell.file.file object at 0x0000028E21155E10>


## Target 객체 찾기

In [173]:
list_of_target = ifcopenshell.util.selector.filter_elements(ifc_file, "My_Data.cnv_class=target")
print(list_of_target)

{#1220=IfcWall('3Grk8JCXL0Wv356Wrvw3Rc',$,'Wall',$,$,#1620,#1228,$,$)}


## Target 객체 위치 좌표점 가져오기

In [174]:
# geom 세팅 생성
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# Target 객체 반복
for element in list_of_target:
    # shape 가져오기
    shape = ifcopenshell.geom.create_shape(settings, element)
    geometry = shape.geometry
    verts = np.array(geometry.verts).reshape(-1,3)
    
    #중심점 구하기
    center = verts.mean(axis=0)
    print('center : ', center)

    # 소수점반올림
    rounded_center = np.round(center, 3)
    print("Rounded center point:", rounded_center)



    

center :  [ 9.99999992e-01 -3.32796162e-09  1.50000000e+00]
Rounded center point: [ 1.  -0.   1.5]


## Target 객체 위치 좌표점으로부터 ray 생성(옵션 : ray개수입력(방사형))

## ray객체를 반복문으로 돌리며 My_Data.cnv_class=distraction 인 객체 찾기

## 전체 ray객체 기준 distraction 간섭 객체개수 % 결과값 확인

# TEST


In [ ]:
import multiprocessing
import ifcopenshell
import ifcopenshell.geom

tree = ifcopenshell.geom.tree()
settings = ifcopenshell.geom.settings()
iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
if iterator.initialize():
    while True:
        # Use triangulation to build a BVH tree
        # tree.add_element(iterator.get())

        # Alternatively, use this code to build an unbalanced binary tree
        tree.add_element(iterator.get_native())

        if not iterator.next():
            break


origin = (0., 0., 0.)
direction = (1., 0., 0.)
results = tree.select_ray(origin, direction, length=50.)
print(results)
for result in results:
    print(ifc_file.by_id(result.instance.id())) # The element the ray intersects with
    print(list(result.position)) # The XYZ intersection point
    print(result.distance) # The distance between the ray origin and the intersection
    print(list(result.normal)) # The normal of the face being intersected
    print(result.dot_product) # The dot product of the face being intersected with the ray



        


(<ifcopenshell.ifcopenshell_wrapper.ray_intersection_result; proxy of <Swig Object of type 'IfcGeom::ray_intersection_result *' at 0x0000028E2128DF20> >, <ifcopenshell.ifcopenshell_wrapper.ray_intersection_result; proxy of <Swig Object of type 'IfcGeom::ray_intersection_result *' at 0x0000028E21371950> >)
#1621=IfcWall('14CTxR01n8AOBlWZYP3AhA',$,'Wall',$,$,#1673,#1635,$,$)
<Swig Object of type 'std::array< double,3 > *' at 0x0000028E2138B840>
1.887776367734931
-0.9999999999999811
#1621=IfcWall('14CTxR01n8AOBlWZYP3AhA',$,'Wall',$,$,#1673,#1635,$,$)
<Swig Object of type 'std::array< double,3 > *' at 0x0000028E2138B840>
1.937776367734932
0.9999999999999811
